In [2]:
# from data.loader import DataLoader
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import datetime
from typing import Dict

# import yfinance as yf

In [3]:
# import yfinance as yf
# import pandas as pd

# # Step 1: Download BTC and ETH data
# tickers = ['BTC-USD', 'ETH-USD']
# data = yf.download(tickers, start='2018-01-01', end='2025-01-01', group_by='ticker')

# # Step 2: Flatten MultiIndex columns
# data.columns = ['_'.join(col).strip() for col in data.columns.values]

# # Step 3: Separate into BTC and ETH DataFrames
# btc_data = data[[col for col in data.columns if 'BTC-USD' in col]].copy()
# eth_data = data[[col for col in data.columns if 'ETH-USD' in col]].copy()

# # Step 4: Clean up column names
# btc_data.columns = [col.replace('BTC-USD_', '') for col in btc_data.columns]
# eth_data.columns = [col.replace('ETH-USD_', '') for col in eth_data.columns]

# # Step 5: Add 'Date' column from index
# btc_data = btc_data.reset_index()
# eth_data = eth_data.reset_index()

# # Step 6: Save to CSV
# btc_data.to_csv('btc_data.csv', index=False)
# eth_data.to_csv('eth_data.csv', index=False)

# print("✅ BTC and ETH data saved to 'btc_data.csv' and 'eth_data.csv'")
# data.reset_index()

fees = 0.0006

In [4]:
BTC = pd.read_csv('btc_data.csv')
BTC.Date = pd.to_datetime(BTC.Date)
BTC

FileNotFoundError: [Errno 2] No such file or directory: 'btc_data.csv'

In [ ]:
BTC_2020 = BTC[BTC.Date.dt.year == 2020].reset_index(drop=True)
BTC_2020

In [ ]:
sns.set(style="darkgrid")

# Create the line plot
plt.figure(figsize=(12, 6))
sns.lineplot(x=BTC_2020.Date, y=BTC_2020.Close)

# Add labels and title
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('BTC Price Over Time')

# Rotate dates for readability if needed
plt.xticks(rotation=45)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
BTC_2020

In [ ]:
netflow = pd.read_csv('netflow.csv')
netflow.Date = pd.to_datetime(netflow.Date, unit='ms').dt.date
netflow.Date = pd.to_datetime(netflow.Date)
netflow.dtypes

In [ ]:
netflow_2020 = netflow[netflow.Date.dt.year == 2020].reset_index(drop=True)

In [ ]:
netflow_2020.drop_duplicates()
netflow_2020

In [ ]:
data = BTC_2020.merge(netflow_2020.NetflowVolume, left_index=True, right_index=True)
data.dtypes

In [ ]:
data['price_change'] = data.Close.pct_change()

In [ ]:
def z_score(data, window):
    roll = data.rolling(window, min_periods=1)
    mean = roll.mean()
    std = roll.std()

    return (mean - data)/ std

data['z_score'] = z_score(data.NetflowVolume, 14)

In [ ]:
import math

def generate_position(data, threshold):
    z = data.z_score
    positions = [0]

    # high netflow volume = sell
    for score in z:
        if math.isnan(score):
            continue
        
        if score > threshold:
            position = -1
        else:
            position = 1
            
        positions.append(position)
        
    positions_series = pd.Series(positions)
    data['positions'] = positions_series  

    data['trades'] = abs(data.positions - data.positions.shift(1))

    return data

position_data = generate_position(data, 1)


In [ ]:
def calc_metrics(data, fees, window):

    p_change = data.price_change
    position = data.positions.shift(1)
    trade = data.trades

    pnl = p_change*position-trade*fees
    data['pnl']= pnl

    data['equity'] = pnl.cumsum()

    avg_pnl = pnl.sum()/pnl.count()
    sharp_ratio = avg_pnl/pnl.std() * math.sqrt(365)
    
    trade_per_interval = data.trades.sum()/ data.trades.count()

    # drawdown = 
    
    currmax = data.equity.cummax()
    daily_drawdown = data.equity - currmax 

    data['drawdown'] = daily_drawdown
    MDD = daily_drawdown.min()

    print(MDD)
    return data, sharp_ratio, trade_per_interval, MDD

calc_data, sharp_ratio, trade_per_interval, MDD = calc_metrics(position_data, fees, 14)

In [ ]:
print(f"Sharp Ratio: {sharp_ratio}\nTrade per Interval: {trade_per_interval}\nMDD: {MDD}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="darkgrid")

# Create figure and primary axis
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot BTC Price (Left Y-axis)
color = 'tab:blue'
sns.lineplot(x=BTC_2020.Date, y=BTC_2020.Close, ax=ax1, color=color, label='BTC Price')
ax1.set_xlabel('Date')
ax1.set_ylabel('BTC Price (USD)', color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Create secondary axis for Equity
ax2 = ax1.twinx()
color = 'tab:red'
sns.lineplot(x=calc_data.Date, y=calc_data.equity, ax=ax2, color=color, label='Strategy Equity')
ax2.set_ylabel('Equity (Scaled)', color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Add title and format
plt.title('BTC Price vs. Strategy Performance (2020)')
fig.legend(loc="upper left", bbox_to_anchor=(0.15, 0.9))

# Rotate dates and adjust layout
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()